## Task 1. Локальное выравнивание

In [1]:
from Bio.SubsMat.MatrixInfo import blosum62 as tabBL
from Bio.SubsMat.MatrixInfo import pam60 as tabPAM
import numpy as np

In [2]:
def alignProteins(seq1, seq2, table, printMatrix=True):
    
    seq1 = seq1.upper()
    seq2 = seq2.upper()
    
    # compute dp matrix
    
    n1, n2 = len(seq1), len(seq2)
    dp = np.empty((n1 + 1, n2 + 1), dtype=int)
    dp[:, 0] = np.zeros((1, n1 + 1))
    dp[0, :] = np.zeros((1, n2 + 1))
    
    gap_penalty = -10

    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            penalty = table[(seq1[i - 1], seq2[j - 1])] if (seq1[i - 1], seq2[j - 1]) in table else \
                        table[(seq2[j - 1], seq1[i - 1])]
            dp[i, j] = max(dp[i - 1, j - 1] + penalty, 
                           dp[i - 1, j] + gap_penalty, 
                           dp[i, j - 1] + gap_penalty,
                           0)
            
    if printMatrix:        
        seq1_list = [''] + list(seq1)
        seq2_list = ['', ''] + list(seq2)
        print("\t\t\tDP matrix:")
        print('\t')
        print("\t".join(seq2_list))
        for i in range(n1 + 1):
            print(seq1_list[i], '\t', '\t'.join(list(dp[i].astype(str))))
            
    # back step
    
    i, j = np.unravel_index(np.argmax(dp), dp.shape)
    end_coord = i, j
    seq1_aligned_list = []
    seq2_aligned_list = []
    
    while dp[i, j] > 0:
        if dp[i - 1, j] + gap_penalty == dp[i, j]:
            i -= 1
            seq1_aligned_list.append(seq1[i])
            seq2_aligned_list.append('-')
        elif dp[i, j - 1] + gap_penalty == dp[i, j]:
            j -= 1
            seq1_aligned_list.append('-')
            seq2_aligned_list.append(seq2[j])
        else:
            i -= 1
            j -= 1
            seq1_aligned_list.append(seq1[i])
            seq2_aligned_list.append(seq2[j])
            
    start_coord = (i, j)
    
    seq1_aligned_list = seq1_aligned_list[::-1]
    seq2_aligned_list = seq2_aligned_list[::-1]
    
    print("\n\t\t\tAlignment:\n")
    
    tab1 = max(0, start_coord[1] - start_coord[0])
    tab2 = max(0, start_coord[0] - start_coord[1])    
    
    seq1_aligned = ''.join([' '] * tab1) + seq1[: start_coord[0]].lower() + \
                   ''.join(seq1_aligned_list).upper() + seq1[end_coord[0]:].lower()
    seq2_aligned = ''.join([' '] * tab2) + seq2[: start_coord[1]].lower() + \
                   ''.join(seq2_aligned_list).upper() + seq2[end_coord[1]:].lower()
    
    len_aligned = len(seq1_aligned_list)
    ticks = []
    for i in range(len_aligned):
        ticks.append('|' if seq1_aligned_list[i] == seq2_aligned_list[i] else ' ')
    
    print(seq1_aligned)
    print(''.join([' '] * max(start_coord) + ticks))
    print(seq2_aligned)
    print("\nWeight =", dp[end_coord])

##### BLOSUM 62

In [13]:
seq1 = "AGCTGATTCGWRNDTSRHFA"
seq2 = "ATCMTRTNCFA"
alignProteins(seq1, seq2, tabBL)

			DP matrix:
	
		A	T	C	M	T	R	T	N	C	F	A
 	 0	0	0	0	0	0	0	0	0	0	0	0
A 	 0	4	0	0	0	0	0	0	0	0	0	4
G 	 0	0	2	0	0	0	0	0	0	0	0	0
C 	 0	0	0	11	1	0	0	0	0	9	0	0
T 	 0	0	5	1	10	6	0	5	0	0	7	0
G 	 0	0	0	2	0	8	4	0	5	0	0	7
A 	 0	4	0	0	1	0	7	4	0	5	0	4
T 	 0	0	9	0	0	6	0	12	4	0	3	0
T 	 0	0	5	8	0	5	5	5	12	3	0	3
C 	 0	0	0	14	7	0	2	4	2	21	11	1
G 	 0	0	0	4	11	5	0	0	4	11	18	11
W 	 0	0	0	0	3	9	2	0	0	2	12	15
R 	 0	0	0	0	0	2	14	4	0	0	2	11
N 	 0	0	0	0	0	0	4	14	10	0	0	1
D 	 0	0	0	0	0	0	0	4	15	7	0	0
T 	 0	0	5	0	0	5	0	5	5	14	5	0
S 	 0	1	1	4	0	1	4	1	6	4	12	6
R 	 0	0	0	0	3	0	6	3	1	3	2	11
H 	 0	0	0	0	0	1	0	4	4	0	2	1
F 	 0	0	0	0	0	0	0	0	1	2	6	0
A 	 0	4	0	0	0	0	0	0	0	1	0	10

			Alignment:

AGCTGATTCgwrndtsrhfa
| |   | |
ATCMTRTNCfa

Weight = 21


##### PAM 60

In [14]:
seq1 = "AGCTGATTCGWRNDTSRHFA"
seq2 = "ATCMTRTNCFA"
alignProteins(seq1, seq2, tabPAM)

			DP matrix:
	
		A	T	C	M	T	R	T	N	C	F	A
 	 0	0	0	0	0	0	0	0	0	0	0	0
A 	 0	5	1	0	0	1	0	1	0	0	0	5
G 	 0	0	2	0	0	0	0	0	0	0	0	0
C 	 0	0	0	11	1	0	0	0	0	9	0	0
T 	 0	1	6	1	9	7	0	6	0	0	3	1
G 	 0	0	0	0	0	6	0	0	5	0	0	3
A 	 0	5	1	0	0	1	1	1	0	0	0	5
T 	 0	1	11	1	0	6	0	7	0	0	0	1
T 	 0	1	7	6	0	6	2	6	6	0	0	1
C 	 0	0	0	16	6	0	0	0	0	15	5	0
G 	 0	0	0	6	10	3	0	0	0	5	8	5
W 	 0	0	0	0	0	1	3	0	0	0	2	0
R 	 0	0	0	0	0	0	9	0	0	0	0	0
N 	 0	0	0	0	0	0	0	8	6	0	0	0
D 	 0	0	0	0	0	0	0	0	10	0	0	0
T 	 0	1	6	0	0	6	0	6	0	5	0	1
S 	 0	1	2	5	0	1	4	1	7	0	0	1
R 	 0	0	0	0	3	0	9	0	0	1	0	0
H 	 0	0	0	0	0	0	0	4	1	0	0	0
F 	 0	0	0	0	0	0	0	0	0	0	8	0
A 	 0	5	1	0	0	1	0	1	0	0	0	13

			Alignment:

agctgaTTCgwrndtsrhfa
       ||
      ATCmtrtncfa

Weight = 16


## Task 2. Множественное выравнивание

Возьмем для выравнивания семейство родственных белков "Adenosine and adenine nucleotide receptors" (из них 7 белков). Воспользуемся сервисом Clustal Omega. Результат выравнивания: